In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import pickle
import helper_functions_06b as hf

In [2]:
DATE = ['20230306', '20230307', '20230314', '20230317', '20230317']
SESSION = ['SE_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SM_Lek1']
DRONE = ['P3D6', 'P1D2', 'P3D6', 'P2D3', 'P2D4']
VIDEOS = ['0663', '0796', '0978', '0410', '0035']

tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'

In [3]:
# Thresholds for translation, rotation, and scale
thresholds = {
    'T_translation': 30,  # e.g., 30 pixels
    'T_rotation': 20,     # e.g., 20 degrees
    'T_scale': 0.1         # e.g., 10% change
}

In [4]:
for date,session,drone,video in zip(DATE,SESSION,DRONE,VIDEOS):
    internal_path = tracking_directory + '/' + date + '/' + session + '/' + drone
    file = internal_path + '/' + date + '_' + session + '_' + drone + '_DJI_' + video + '_YOLO_tracked.csv'
    
    print(date, session, drone, os.path.basename(file))
    
    df = pd.read_csv(file)
    df['x'] = df['bb_left'] + df['bb_width']/2
    df['y'] = df['bb_top'] + df['bb_height']/2
    df['idx'] = df['track_id']

    df = df.loc[:,['frame', 'x', 'y', 'idx']]

    # Step 1: Identify anchor frames
    anchor_frames = hf.identify_anchor_frames(df, thresholds)

    # Step 2: Find the best anchor for each frame
    df = hf.find_best_anchor_for_each_frame_parallel(df, anchor_frames)

    # Step 3: Apply homography using the best anchor for each frame
    df, homography_matrices, rms_errors = hf.apply_homography_to_best_anchor(df)

    anchor_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_Anchored.csv')
    pickle_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_homographies.pkl')
    df.to_csv(os.path.join(os.path.dirname(file), anchor_filename), index=False, mode='w')
    with open(os.path.join(os.path.dirname(file), pickle_filename), 'wb') as f:
        pickle.dump(homography_matrices, f)

20230306 SE_Lek1 P3D6 20230306_SE_Lek1_P3D6_DJI_0663_YOLO_tracked.csv


KeyboardInterrupt: 